In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

In [ ]:
# Download of .zip file
url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/2d3kr8zynf-4.zip"
output = f"./raw_data/dataset.zip"
subprocess.run(["wget", "--no-check-certificate", url, "-O", output])

# Extraction of .zip file
subprocess.run(["unzip", output])
subprocess.run(["mv", "/raw_data/2d3kr8zynf-4", "/raw_data/arbovirus_clinical_data"])

In [ ]:
df = pd.read_csv('/content/arbovirus_clinical_data/data set.csv')